In [1]:
from tensorflow import keras

# Load the model
model = keras.models.load_model('model.h5')


In [2]:
# pip install opencv-python

In [3]:
import cv2
import numpy as np

# Load and preprocess the image
image_path = 'Datacluster Auto (138).jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224))  # Resize the image to match your model's input size
image = image / 255.0  # Normalize the pixel values

# Make predictions
predictions = model.predict(np.expand_dims(image, axis=0))  # Add a batch dimension
print(predictions)
# The 'predictions' variable now contains the predicted probabilities for each class.
# You can use np.argmax(predictions) to find the index of the class with the highest probability.

# For example, if you want to find the class with the highest probability:
predicted_class = np.argmax(predictions)

# You might also want to decode the class index to get the class label based on your dataset.
class_labels = ["Auto", "Bus", "Tempo", "Tractor", "Truck"]
predicted_label = class_labels[predicted_class]

# Print the predicted class label
print("Predicted Class:", predicted_label)

1/1 [==============================] - 0s 488ms/step
[[8.8633072e-01 4.2508165e-03 8.3982630e-04 1.0027897e-01 8.2997074e-03]]
Predicted Class: Auto


In [4]:
# pip install cvlib

In [5]:
# pip install --upgrade pip

In [6]:
# pip install vidgear

In [7]:
# pip install opencv-contrib-python

In [8]:
# # from yolov3
# import cv2
# import cvlib as cv
# from cvlib.object_detection import draw_bbox
# from vidgear.gears import CamGear    
# cap = cv2.VideoCapture("Vehicle_count_test.mp4")
# count=0
# label_file_path = 'Untitled Folder/yolov3.txt'
# cfg_file_path = 'Untitled Folder/yolov3.cfg'
# while True:
#     ret, frame = cap.read()
#     count += 1
#     if count % 6 != 0:
#         continue
#     frame=cv2.resize(frame,(1020,600))
#     # bbox,label,conf = cv.detect_common_objects(frame,
#     # confidence=0.5,
#     # enable_gpu=True,
#     # model='yolov4')
#     # frame = draw_bbox(frame,bbox,label = label_file_path ,cfg = cfg_file_path)
#     cv2.imshow("FRAME",frame)
#     if cv2.waitKey(1)&0xFF==27:
#         break
# cap.release()
# cv2.destroyAllWindows()

In [9]:
# pip install opencv-contrib-python

In [18]:
import cv2
import numpy as np
from time import sleep
width_min=80 #Largura minima do retangulo
height_min=80 #Altura minima do retangulo

offset=6 #Erro permitido entre pixel  

pos=400 #Posição da linha de contagem 

delay= 60 #FPS do vídeo

detect = []
a =0
b =0
te = 0
ta =0
tu =0
o =0
count =0
	
def page_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

cap = cv2.VideoCapture('Vehicle_count_test.mp4')
subtractor = cv2.createBackgroundSubtractorMOG2()

def pred(image,x,y,w,h):
    if w <= 0 or h <= 0:
        return "Invalid"

    roi = image[y:h, x:w]
    if roi.size == 0:
        return "Empty"
    # image = cv2.imread(image_path)
    # roi = image[y:h,x:w]
    resized_roi = cv2.resize(roi,(224,224))  # Resize the image to match your model's input size
    image = image / 255.0  # Normalize the pixel values
    model.predict(np.expand_dims(image, axis=0))
    predicted_class = np.argmax(predictions)
    # You might also want to decode the class index to get the class label based on your dataset.
    class_labels = ["Auto", "Bus", "Tempo", "Tractor", "Truck"]
    predicted_label = class_labels[predicted_class]
    return predicted_label

while True:
    ret , frame1 = cap.read()
    if not ret:
        break
    frame1 = cv2.resize(frame1,(1020,600))
    # tempo = float(1/delay)
    # sleep(tempo) 
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    img_sub = subtractor.apply(blur)
    dilate = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx (dilate, cv2. MORPH_CLOSE , kernel)
    dilatada = cv2.morphologyEx (dilatada, cv2. MORPH_CLOSE , kernel)
    cont,h=cv2.findContours(dilatada,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cv2.line(frame1, (25, pos), (1800, pos), (255,127,0), 3) 
    for(i,c) in enumerate(cont):
        (x,y,w,h) = cv2.boundingRect(c)
        valid_cont = (w >= width_min) and (h >= height_min)
        if not valid_cont:
            continue

        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)        
        centre = page_center(x, y, w, h)
        detect.append(centre)
        cv2.circle(frame1, centre, 4, (0, 0,255), -1)
        
        for (x,y) in detect:
            if y<(pos+offset) and y>(pos-offset):
                count+=1
                cv2.line(frame1, (25, pos), (1000, pos), (0,127,255), 3)  
                detect.remove((x,y))
                vehicle_type = pred(frame1, x, y, w, h)
                if vehicle_type == "Auto":
                    a += 1
                elif vehicle_type == "Bus":
                    b += 1
                elif vehicle_type == "Tempo":
                    te += 1
                elif vehicle_type == "Tractor":
                    ta += 1
                elif vehicle_type == "Truck":
                    tu += 1
                else:
                    o += 1
                
                # print("car is detected : "+str(count))       
    # cv2.putText(frame1, "AUTO COUNT : "+str(a), (100, 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    # cv2.putText(frame1, "BUS COUNT : "+str(b), (100, 30), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    # cv2.putText(frame1, "TEMPO COUNT : "+str(te), (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    # cv2.putText(frame1, "TRACTOR COUNT : "+str(ta), (100, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    # cv2.putText(frame1, "TRUCK COUNT : "+str(tu), (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    cv2.putText(frame1, "VEHICLES COUNT : "+str(count), (100, 110), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255),5)
    cv2.imshow("Video Original" , frame1)
    cv2.imshow("Detectar",dilatada)

    if cv2.waitKey(1) == 27:
        break
print("Total Vehicles Count: ",count)
print("Auto count :",a)
print("Bus count :",b)
print("Tempo count :",te)
print("Tractor count :",ta)
print("Truck count :",tu)
print("Other Vehicles count :",o)
cv2.destroyAllWindows()
cap.release()


1/1 [==============================] - 1s 1s/step
Total Vehicles Count:  25
Auto count : 2
Bus count : 0
Tempo count : 0
Tractor count : 0
Truck count : 0
Other Vehicles count : 23
